In [20]:
import pandas as pd


AAPL


,title,source,author,date,length,wordcount,totalsearchresults,relevancescore,contentcomponentid,vendorreportid,docid
0,INTERNATIONAL PATENT: APPLE INC. FILES APPLICA...,US Fed News,NaN,2021-01-01,262 words,262,10,62,282801,USFDNW2021,urn:contentItem:61N5-PKX1-F12F-F1DG-00000-00
1,INTERNATIONAL PATENT: APPLE INC. FILES APPLICA...,US Fed News,NaN,2021-01-01,285 words,285,10,57,282801,USFDNW2021,urn:contentItem:61N8-CWB1-F12F-F3VS-00000-00
2,"Apple Hospitality REIT, Inc. ( NYSE :APLE) Sha...",Zolmax.com,NaN,2021-01-01,795 words,795,10,50,484302,AMCN-116308,urn:contentItem:61NB-DBD1-JCMN-Y0BG-00000-00
3,"Apple ( NASDAQ :AAPL) Cut to ""Buy"" at BidaskClub",American Banking and Market News,NaN,2021-01-01,995 words,995,10,30,439934,AMBNWS2021,urn:contentItem:61NC-6WB1-JB40-X3PR-00000-00
4,"Apple, Best Stock Of 2020, Surges Into New Year",24/7 Wall St.,Douglas A. McIntyre,2021-01-01,669 words,669,10,25,484220,TFS-5001,urn:contentItem:61NB-DBD1-JCMN-Y1XM-00000-00


In [36]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from collections import Counter
import os
import matplotlib.pyplot as plt

tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer, truncation=True, padding=True)

def process_content(content):
    tokens = tokenizer.tokenize(content)
    max_length = 512 - tokenizer.num_special_tokens_to_add()
    token_chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    
    sentiments = []
    for chunk in token_chunks:
        chunk_text = tokenizer.convert_tokens_to_string(chunk)
        part_sentiment = nlp(chunk_text[0:513])

        sentiments.extend(part_sentiment)

    # Aggregate sentiments by majority voting
    label_counts = Counter([sentiment['label'] for sentiment in sentiments])
    most_common_label, _ = label_counts.most_common(1)[0]
    
    # Dynamically compute average scores for a more nuanced aggregation
    scores = {}
    for sentiment in sentiments:
        label = sentiment['label']
        if label in scores:
            scores[label] += sentiment['score']
        else:
            scores[label] = sentiment['score']
    
    for label in scores.keys():
        scores[label] /= len(token_chunks)
    print(scores)
    final_label = max(scores, key=scores.get)

    if final_label == 'Positive':
        value = 3
    elif final_label == 'Neutral':
        value = 2
    elif final_label == 'Negative':
        value = 1
    
    return {'label': final_label, 'score': scores[final_label], 'value' : value}

def processData(df):
    results = []
    for content in df['title']:
        processed_result = process_content(content)
        results.append(processed_result)

    df['Sentiment'] = results
    return df

def labelExtract(df):
    # Extract 'label', 'score', and 'value' keys into new columns
    df['Sentiment_Label'] = df['Sentiment'].apply(lambda x: x.get('label', 'N/A'))
    df['Sentiment_Score'] = df['Sentiment'].apply(lambda x: x.get('score', 0))
    df['Sentiment_Value'] = df['Sentiment'].apply(lambda x: x.get('value', 'N/A'))
    
    return df

def sentimentIndexByDay(df, companyName):
    df['Sentiment_index'] = df['Sentiment_Score'] * df['Sentiment_Value']
    sentiment_index_by_day = df.groupby('date')['Sentiment_index'].mean()
    
    sentiment_index_by_day.to_csv(f'{companyName}_sentiment_index.csv')

    return sentiment_index_by_day


def Plot(sentiment_index_by_day, companyName):
    # Plotting the sentiment index by day
    plt.figure(figsize=(18, 10))
    plt.plot(sentiment_index_by_day.index, sentiment_index_by_day.values)
    plt.xlabel('Date')
    plt.ylabel('Sentiment Index')
    plt.title(f'Sentiment Index for {companyName} by Day')
    plt.xticks(rotation=45)
    plt.xticks(range(0, len(sentiment_index_by_day.index), 10), sentiment_index_by_day.index[::10])
    plt.show()


In [31]:
companyName = "newsdata\AAPL_21-01_24-04_news"
companyName = companyName.split('_')[0]
print(companyName)
print(companyName.split('\\')[1])

newsdata\AAPL
AAPL


In [ ]:
import glob
# Get a list of all CSV files in the "newsdata" folder
csv_files = glob.glob('newsdata/*.csv')

for file_name in csv_files:
    companyName = file_name.split('_')[0]
    companyName = companyName.split('\\')[1]
    print(companyName)

    df = pd.read_csv(file_name)

    df = processData(df)
    df = labelExtract(df)
    dayData = sentimentIndexByDay(df, companyName)
    Plot(dayData, companyName)


Moving Average

,date,Sentiment_index,Moving_Average
0,2021-01-01,2.169724,NaN
1,2021-01-02,2.313737,NaN
2,2021-01-03,2.043799,NaN
3,2021-01-04,1.837105,NaN
4,2021-01-05,1.970936,NaN
...,...,...,...
556,2024-03-27,2.107532,2.037936
557,2024-03-28,1.789584,2.027496
558,2024-03-29,1.991708,2.027458
559,2024-03-30,1.951463,2.026721


In [36]:
import pandas as pd

df = pd.read_csv('AAPL_sentiment_index.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

# Calculate the moving average with a window size of 7
df['MA7'] = df['Sentiment_index'].rolling(window=7).mean()
df['MA14'] = df['Sentiment_index'].rolling(window=14).mean()
df['MA31'] = df['Sentiment_index'].rolling(window=31).mean()
display(df.tail())
df = df.dropna()

df.info()
df.reset_index(inplace=True,drop=True)

,Sentiment_index,MA7,MA14,MA31
date,,,,
2024-03-27,2.107532,2.002024,2.056405,2.037936
2024-03-28,1.789584,1.972104,2.028544,2.027496
2024-03-29,1.991708,1.955748,2.026370,2.027458
2024-03-30,1.951463,1.973768,2.014282,2.026721
2024-03-31,2.025257,1.995227,1.995015,2.027864


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 531 entries, 2021-01-31 to 2024-03-31
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sentiment_index  531 non-null    float64
 1   MA7              531 non-null    float64
 2   MA14             531 non-null    float64
 3   MA31             531 non-null    float64
dtypes: float64(4)
memory usage: 20.7 KB
